In [ ]:
import pandas as pd
import sys, os
sys.path.append(r"C:\Users\Valentino\Documents\MATLAB\PNN_wholeBrain\analysis")

# Custom packages
import dataIO
import AbaTool

# You can provide a "structures.json" file. If you dont, it will download one
structuresFile = r"C:\Users\Valentino\Documents\MATLAB\PNN_wholeBrain\analysis\structures.json"
A = AbaTool.Atlas(nodes=structuresFile)
DFM = AbaTool.AnatomyDataFrameManager(A)


# Load data

In [ ]:
searchpath =  r'D:\proj_PNN-Atlas\RESULTS\colocalization'

dots_df = dataIO.allMiceDots(searchpath)
dots_df = DFM.multiIndexDf_from_dotsDf(dots_df)


# High Resolution

In [ ]:
fine = DFM.dotsRegionsDf_to_fine(dots_df, verbose=False, numberOfCells = True)\
    .xs('CTR', axis=1, level='treat')
fine = DFM.dots_leaveOneOut_correction(fine, min_num_mice=3, min_num_pv=2,min_num_pnn=2)
# Add acronyms to the index
fineAcros = A.ids_to_acronyms(fine.index.get_level_values('fine'))
midAcros = A.ids_to_acronyms(fine.index.get_level_values('mid'))
coarseAcros = A.ids_to_acronyms(fine.index.get_level_values('coarse'))
fine['fine_acro'] = fineAcros
fine['mid_acro'] = midAcros
fine['coarse_acro'] = coarseAcros
fine = fine.set_index(['fine_acro','mid_acro','coarse_acro'], append=True)
# Reorder index in the proper way
fine = fine.reorder_levels(['coarse','coarse_acro','mid','mid_acro','fine','fine_acro'])

fine_avg = fine.groupby('params',axis=1).mean()
fine

# Mid Resolution

In [ ]:
mid = DFM.dotsRegionsDf_to_mid(dots_df, verbose=False, numberOfCells=True)\
    .xs('CTR', axis=1, level='treat')
mid = DFM.dots_leaveOneOut_correction(mid, min_num_mice=3, min_num_pv=2,min_num_pnn=2)
# Add acronyms to the index
midAcros = A.ids_to_acronyms(mid.index.get_level_values('mid'))
coarseAcros = A.ids_to_acronyms(mid.index.get_level_values('coarse'))
mid['mid_acro'] = midAcros
mid['coarse_acro'] = coarseAcros
mid = mid.set_index('mid_acro', append=True)
mid = mid.set_index('coarse_acro', append=True)
# Reorder index in the proper way
mid = mid.reorder_levels(['coarse','coarse_acro','mid','mid_acro'])
mid_avg = mid.groupby('params',axis=1).mean()
mid

# Coarse resolution

In [ ]:
coarse = DFM.dotsRegionsDf_to_coarse(dots_df, verbose=False)\
    .xs('CTR', axis=1, level='treat')

# Add acronyms to the index
coarseAcros = A.ids_to_acronyms(coarse.index.get_level_values('coarse'))
coarse['coarse_acro'] = coarseAcros
coarse = coarse.set_index('coarse_acro', append=True)

coarse_avg = coarse.groupby('params',axis=1).mean()
coarse

In [ ]:
with pd.ExcelWriter('data_SD3.xlsx') as writer:
    coarse.to_excel(writer, sheet_name='coarse', na_rep='NA')
    coarse_avg.to_excel(writer, sheet_name='coarse_miceAvg', na_rep='NA')
    mid.to_excel(writer, sheet_name='medium', na_rep='NA')
    mid_avg.to_excel(writer, sheet_name='medium_expUnitAvg', na_rep='NA')
    fine.to_excel(writer, sheet_name='fine', na_rep='NA')
    fine_avg.to_excel(writer, sheet_name='fine_expUnitAvg', na_rep='NA')